In [27]:
import torch
scores= torch.randn((4, 8))
print(scores)

tensor([[ 0.9094,  0.5632, -1.2995, -0.0162, -0.7846, -0.7688, -0.8744, -0.6386],
        [-0.9376,  0.1207, -0.6128, -0.3607, -0.0858,  0.4954, -1.6601,  0.9203],
        [ 0.7702,  0.3170,  1.4561, -1.0591, -1.7216, -1.1876, -0.1930,  0.2659],
        [ 0.3682, -0.6647,  1.3395, -1.4207, -1.1318, -0.8974, -0.0921, -0.0366]])


In [28]:
topk_logits, topk_indices = torch.topk(scores, 2, dim=1)
print(topk_logits )
print(topk_indices)

tensor([[0.9094, 0.5632],
        [0.9203, 0.4954],
        [1.4561, 0.7702],
        [1.3395, 0.3682]])
tensor([[0, 1],
        [7, 5],
        [2, 0],
        [2, 0]])


In [29]:
def one_hot_with_dtype(data, num_classes, dtype):
    result = torch.zeros([data.size(0), num_classes], device=data.device, dtype=dtype)
    result.scatter_(1, data.unsqueeze(-1), 1)
    return result

indices_s = [x.view(-1) for x in topk_indices.chunk(2, dim=1)]
print(indices_s)
mask_se = [one_hot_with_dtype(x, num_classes=8, dtype=x.dtype) for x in indices_s]
for mask in mask_se:
    print(mask)

[tensor([0, 7, 2, 2]), tensor([1, 5, 0, 0])]
tensor([[1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0]])
tensor([[0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0]])


In [30]:
import torch.nn.functional as F
gates = F.softmax(scores, dim=1)
print(gates)
print(gates.max(dim=1))
importance_scores = -1 * gates.max(dim=1)[0]
print(importance_scores)

tensor([[0.3373, 0.2386, 0.0370, 0.1337, 0.0620, 0.0630, 0.0567, 0.0717],
        [0.0488, 0.1407, 0.0676, 0.0870, 0.1145, 0.2047, 0.0237, 0.3131],
        [0.2003, 0.1273, 0.3978, 0.0322, 0.0166, 0.0283, 0.0765, 0.1210],
        [0.1676, 0.0596, 0.4426, 0.0280, 0.0374, 0.0473, 0.1057, 0.1118]])
torch.return_types.max(
values=tensor([0.3373, 0.3131, 0.3978, 0.4426]),
indices=tensor([0, 7, 2, 2]))
tensor([-0.3373, -0.3131, -0.3978, -0.4426])


In [33]:
mask = mask_se[0]
print(mask)
print(importance_scores.argsort(dim=0))
sorted_mask = mask[importance_scores.argsort(dim=0)]
print(sorted_mask)
re_sorted_mask = sorted_mask[importance_scores.argsort(dim=0).argsort(0)]
print(re_sorted_mask)


tensor([[1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0]])
tensor([3, 2, 0, 1])
tensor([[0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1]])
tensor([[1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0]])
